In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Dense
import xgboost as xgb
from sklearn.metrics import mean_squared_error, precision_score, recall_score
import warnings

warnings.filterwarnings('ignore')

def hybrid_recsys_load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def hybrid_recsys_preprocess_data(data):
    data['Brand_Name'] = data['Brand_Name'].fillna('')
    user_encoder = LabelEncoder()
    item_encoder = LabelEncoder()
    data['user_id'] = user_encoder.fit_transform(data['Client_ID'])
    data['item_id'] = item_encoder.fit_transform(data['Product_ID'])
    return data, user_encoder, item_encoder

def hybrid_recsys_aggregate_data(data):
    data_aggregated = data.groupby(['user_id', 'item_id'], as_index=False)['Monetary_Value'].sum()
    return data_aggregated

def hybrid_recsys_create_interaction_matrix(data_aggregated):
    interaction_matrix = data_aggregated.pivot(index='user_id', columns='item_id', values='Monetary_Value').fillna(0)
    return interaction_matrix

def hybrid_recsys_train_test_split(interaction_matrix):
    train_data, test_data = train_test_split(interaction_matrix, test_size=0.2, random_state=42)
    return train_data, test_data

def hybrid_recsys_train_autoencoder(train_data, test_data, input_dim, encoding_dim=10):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(train_data, train_data, epochs=10, batch_size=128, shuffle=True, validation_data=(test_data, test_data), verbose=1)
    encoder = Model(input_layer, encoded)
    encoded_train_data = encoder.predict(train_data)
    return encoded_train_data, encoder

def hybrid_recsys_train_xgboost(encoded_train_data, train_data):
    x_train = encoded_train_data
    y_train = train_data.values
    if y_train.ndim == 1:
        y_train = y_train.reshape(-1, 1)
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10, max_depth=5, learning_rate=0.1)
    model.fit(x_train, y_train)
    return model

def hybrid_recsys_generate_recommendations(user_name, user_encoder, encoded_train_data, model, item_encoder, interaction_matrix, data):
    user_id = user_encoder.transform([user_name])[0]
    user_vector = encoded_train_data[user_id].reshape(1, -1)
    predicted_ratings = model.predict(user_vector).flatten()
    assert len(predicted_ratings) == interaction_matrix.shape[1], "Predicted ratings length mismatch."
    recommendations = pd.DataFrame({'item_id': range(len(predicted_ratings)), 'predicted_rating': predicted_ratings})
    top_10_recommendations = recommendations.sort_values(by='predicted_rating', ascending=False).head(10)
    top_10_recommendations['Product_ID'] = item_encoder.inverse_transform(top_10_recommendations['item_id'])
    top_10_recommendations = top_10_recommendations.merge(data[['Product_ID', 'Product_Name']].drop_duplicates(), on='Product_ID')
    return top_10_recommendations

def hybrid_recsys_evaluate_recommendations(user_name, user_encoder, item_encoder, interaction_matrix, model, data, k=10):
    user_id = user_encoder.transform([user_name])[0]
    true_ratings = interaction_matrix.loc[user_id].values
    user_vector = encoded_train_data[user_id].reshape(1, -1)
    predicted_ratings = model.predict(user_vector).flatten()
    
    true_top_k = np.argsort(true_ratings)[-k:]
    pred_top_k = np.argsort(predicted_ratings)[-k:]

    true_positives = np.intersect1d(true_top_k, pred_top_k)
    recall = len(true_positives) / len(true_top_k)
    precision = len(true_positives) / len(pred_top_k)
    mse = mean_squared_error(true_ratings, predicted_ratings)
    
    return recall, precision, mse

def main():
    file_path = "/Users/igormol/Desktop/Analytics_Fusion/sample/Marajo.csv"
    data = hybrid_recsys_load_data(file_path)
    data, user_encoder, item_encoder = hybrid_recsys_preprocess_data(data)
    data_aggregated = hybrid_recsys_aggregate_data(data)
    interaction_matrix = hybrid_recsys_create_interaction_matrix(data_aggregated)
    train_data, test_data = hybrid_recsys_train_test_split(interaction_matrix)
    input_dim = interaction_matrix.shape[1]
    encoded_train_data, encoder = hybrid_recsys_train_autoencoder(train_data, test_data, input_dim)
    model = hybrid_recsys_train_xgboost(encoded_train_data, train_data)
    
    users = interaction_matrix.index.tolist()
    recalls, precisions, mses = [], [], []
    
    for user_id in users:
        user_name = user_encoder.inverse_transform([user_id])[0]
        recall, precision, mse = hybrid_recsys_evaluate_recommendations(user_name, user_encoder, item_encoder, interaction_matrix, model, data)
        recalls.append(recall)
        precisions.append(precision)
        mses.append(mse)
        print(f"User: {user_name}, Recall@10: {recall:.4f}, Precision@10: {precision:.4f}, MSE: {mse:.4f}")
    
    print(f"\nMean Recall@10: {np.mean(recalls):.4f}")
    print(f"Mean Precision@10: {np.mean(precisions):.4f}")
    print(f"Overall MSE: {np.mean(mses):.4f}")

if __name__ == "__main__":
    main()


Epoch 1/10
76/76 [==============================] - 2s 19ms/step - loss: 2934.2261 - val_loss: 2532.0327
Epoch 2/10
76/76 [==============================] - 1s 14ms/step - loss: 2934.0688 - val_loss: 2531.9919
Epoch 3/10
76/76 [==============================] - 1s 16ms/step - loss: 2934.0371 - val_loss: 2531.9739
Epoch 4/10
76/76 [==============================] - 1s 17ms/step - loss: 2934.0227 - val_loss: 2531.9670
Epoch 5/10
76/76 [==============================] - 1s 14ms/step - loss: 2934.0149 - val_loss: 2531.9626
Epoch 6/10
76/76 [==============================] - 1s 14ms/step - loss: 2934.0090 - val_loss: 2531.9604
Epoch 7/10
76/76 [==============================] - 1s 12ms/step - loss: 2934.0046 - val_loss: 2531.9543
Epoch 8/10
76/76 [==============================] - 1s 12ms/step - loss: 2934.0032 - val_loss: 2531.9536
Epoch 9/10
76/76 [==============================] - 1s 13ms/step - loss: 2934.0012 - val_loss: 2531.9524
Epoch 10/10
303/303 [==============================] - 

NameError: name 'encoded_train_data' is not defined

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Dense
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')

def hybrid_recsys_load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def hybrid_recsys_preprocess_data(data):
    data['Brand_Name'] = data['Brand_Name'].fillna('')
    user_encoder = LabelEncoder()
    item_encoder = LabelEncoder()
    data['user_id'] = user_encoder.fit_transform(data['Client_ID'])
    data['item_id'] = item_encoder.fit_transform(data['Product_ID'])
    return data, user_encoder, item_encoder

def hybrid_recsys_aggregate_data(data):
    data_aggregated = data.groupby(['user_id', 'item_id'], as_index=False)['Monetary_Value'].sum()
    return data_aggregated

def hybrid_recsys_create_interaction_matrix(data_aggregated):
    interaction_matrix = data_aggregated.pivot(index='user_id', columns='item_id', values='Monetary_Value').fillna(0)
    return interaction_matrix

def hybrid_recsys_train_test_split(interaction_matrix):
    train_data, test_data = train_test_split(interaction_matrix, test_size=0.2, random_state=42)
    return train_data, test_data

def hybrid_recsys_train_autoencoder(train_data, test_data, input_dim, encoding_dim=10):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(train_data, train_data, epochs=10, batch_size=128, shuffle=True, validation_data=(test_data, test_data), verbose=1)
    encoder = Model(input_layer, encoded)
    encoded_train_data = encoder.predict(train_data)
    return encoded_train_data, encoder

def hybrid_recsys_train_xgboost(encoded_train_data, train_data):
    x_train = encoded_train_data
    y_train = train_data.values
    if y_train.ndim == 1:
        y_train = y_train.reshape(-1, 1)
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10, max_depth=5, learning_rate=0.1)
    model.fit(x_train, y_train)
    return model

def hybrid_recsys_generate_recommendations(user_name, user_encoder, encoded_train_data, model, item_encoder, interaction_matrix, data):
    user_id = user_encoder.transform([user_name])[0]
    user_vector = encoded_train_data[user_id].reshape(1, -1)
    predicted_ratings = model.predict(user_vector).flatten()
    assert len(predicted_ratings) == interaction_matrix.shape[1], "Predicted ratings length mismatch."
    recommendations = pd.DataFrame({'item_id': range(len(predicted_ratings)), 'predicted_rating': predicted_ratings})
    top_10_recommendations = recommendations.sort_values(by='predicted_rating', ascending=False).head(10)
    top_10_recommendations['Product_ID'] = item_encoder.inverse_transform(top_10_recommendations['item_id'])
    top_10_recommendations = top_10_recommendations.merge(data[['Product_ID', 'Product_Name']].drop_duplicates(), on='Product_ID')
    return top_10_recommendations

def hybrid_recsys_evaluate_recommendations(user_name, user_encoder, item_encoder, interaction_matrix, model, data, encoded_train_data, k=10):
    user_id = user_encoder.transform([user_name])[0]
    true_ratings = interaction_matrix.loc[user_id].values
    user_vector = encoded_train_data[user_id].reshape(1, -1)
    predicted_ratings = model.predict(user_vector).flatten()
    
    true_top_k = np.argsort(true_ratings)[-k:]
    pred_top_k = np.argsort(predicted_ratings)[-k:]

    true_positives = np.intersect1d(true_top_k, pred_top_k)
    recall = len(true_positives) / len(true_top_k)
    precision = len(true_positives) / len(pred_top_k)
    mse = mean_squared_error(true_ratings, predicted_ratings)
    
    return recall, precision, mse

def main():
    file_path = "/Users/igormol/Desktop/Analytics_Fusion/sample/Marajo.csv"
    data = hybrid_recsys_load_data(file_path)
    data, user_encoder, item_encoder = hybrid_recsys_preprocess_data(data)
    data_aggregated = hybrid_recsys_aggregate_data(data)
    interaction_matrix = hybrid_recsys_create_interaction_matrix(data_aggregated)
    train_data, test_data = hybrid_recsys_train_test_split(interaction_matrix)
    input_dim = interaction_matrix.shape[1]
    encoded_train_data, encoder = hybrid_recsys_train_autoencoder(train_data, test_data, input_dim)
    model = hybrid_recsys_train_xgboost(encoded_train_data, train_data)
    
    users = interaction_matrix.index.tolist()
    recalls, precisions, mses = []

    for user_id in users:
        user_name = user_encoder.inverse_transform([user_id])[0]
        recall, precision, mse = hybrid_recsys_evaluate_recommendations(user_name, user_encoder, item_encoder, interaction_matrix, model, data, encoded_train_data)
        recalls.append(recall)
        precisions.append(precision)
        mses.append(mse)
        print(f"User: {user_name}, Recall@10: {recall:.4f}, Precision@10: {precision:.4f}, MSE: {mse:.4f}")
    
    print(f"\nMean Recall@10: {np.mean(recalls):.4f}")
    print(f"Mean Precision@10: {np.mean(precisions):.4f}")
    print(f"Overall MSE: {np.mean(mses):.4f}")

if __name__ == "__main__":
    main()


Epoch 1/10
76/76 [==============================] - 2s 19ms/step - loss: 2934.2256 - val_loss: 2532.0232
Epoch 2/10
76/76 [==============================] - 1s 15ms/step - loss: 2934.0652 - val_loss: 2531.9841
Epoch 3/10
76/76 [==============================] - 1s 16ms/step - loss: 2934.0374 - val_loss: 2531.9719
Epoch 4/10
76/76 [==============================] - 1s 17ms/step - loss: 2934.0261 - val_loss: 2531.9617
Epoch 5/10
76/76 [==============================] - 1s 16ms/step - loss: 2934.0188 - val_loss: 2531.9563
Epoch 6/10
76/76 [==============================] - 2s 21ms/step - loss: 2934.0127 - val_loss: 2531.9570
Epoch 7/10
76/76 [==============================] - 1s 19ms/step - loss: 2934.0095 - val_loss: 2531.9563
Epoch 8/10
76/76 [==============================] - 2s 23ms/step - loss: 2934.0066 - val_loss: 2531.9539
Epoch 9/10
76/76 [==============================] - 2s 32ms/step - loss: 2934.0037 - val_loss: 2531.9526
Epoch 10/10
303/303 [==============================] - 

ValueError: not enough values to unpack (expected 3, got 0)

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Dense
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')

def hybrid_recsys_load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def hybrid_recsys_preprocess_data(data):
    data['Brand_Name'] = data['Brand_Name'].fillna('')
    user_encoder = LabelEncoder()
    item_encoder = LabelEncoder()
    data['user_id'] = user_encoder.fit_transform(data['Client_ID'])
    data['item_id'] = item_encoder.fit_transform(data['Product_ID'])
    return data, user_encoder, item_encoder

def hybrid_recsys_aggregate_data(data):
    data_aggregated = data.groupby(['user_id', 'item_id'], as_index=False)['Monetary_Value'].sum()
    return data_aggregated

def hybrid_recsys_create_interaction_matrix(data_aggregated):
    interaction_matrix = data_aggregated.pivot(index='user_id', columns='item_id', values='Monetary_Value').fillna(0)
    return interaction_matrix

def hybrid_recsys_train_test_split(interaction_matrix):
    train_data, test_data = train_test_split(interaction_matrix, test_size=0.2, random_state=42)
    return train_data, test_data

def hybrid_recsys_train_autoencoder(train_data, test_data, input_dim, encoding_dim=10):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoder.fit(train_data, train_data, epochs=10, batch_size=128, shuffle=True, validation_data=(test_data, test_data), verbose=1)
    encoder = Model(input_layer, encoded)
    encoded_train_data = encoder.predict(train_data)
    return encoded_train_data, encoder

def hybrid_recsys_train_xgboost(encoded_train_data, train_data):
    x_train = encoded_train_data
    y_train = train_data.values
    if y_train.ndim == 1:
        y_train = y_train.reshape(-1, 1)
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10, max_depth=5, learning_rate=0.1)
    model.fit(x_train, y_train)
    return model

def hybrid_recsys_generate_recommendations(user_name, user_encoder, encoded_train_data, model, item_encoder, interaction_matrix, data):
    user_id = user_encoder.transform([user_name])[0]
    user_vector = encoded_train_data[user_id].reshape(1, -1)
    predicted_ratings = model.predict(user_vector).flatten()
    assert len(predicted_ratings) == interaction_matrix.shape[1], "Predicted ratings length mismatch."
    recommendations = pd.DataFrame({'item_id': range(len(predicted_ratings)), 'predicted_rating': predicted_ratings})
    top_10_recommendations = recommendations.sort_values(by='predicted_rating', ascending=False).head(10)
    top_10_recommendations['Product_ID'] = item_encoder.inverse_transform(top_10_recommendations['item_id'])
    top_10_recommendations = top_10_recommendations.merge(data[['Product_ID', 'Product_Name']].drop_duplicates(), on='Product_ID')
    return top_10_recommendations

def hybrid_recsys_evaluate_recommendations(user_name, user_encoder, item_encoder, interaction_matrix, model, data, encoded_train_data, k=10):
    user_id = user_encoder.transform([user_name])[0]
    true_ratings = interaction_matrix.loc[user_id].values
    user_vector = encoded_train_data[user_id].reshape(1, -1)
    predicted_ratings = model.predict(user_vector).flatten()
    
    true_top_k = np.argsort(true_ratings)[-k:]
    pred_top_k = np.argsort(predicted_ratings)[-k:]

    true_positives = np.intersect1d(true_top_k, pred_top_k)
    recall = len(true_positives) / len(true_top_k)
    precision = len(true_positives) / len(pred_top_k)
    mse = mean_squared_error(true_ratings, predicted_ratings)
    
    return recall, precision, mse

def main():
    file_path = "/Users/igormol/Desktop/Analytics_Fusion/sample/Marajo.csv"
    data = hybrid_recsys_load_data(file_path)
    data, user_encoder, item_encoder = hybrid_recsys_preprocess_data(data)
    data_aggregated = hybrid_recsys_aggregate_data(data)
    interaction_matrix = hybrid_recsys_create_interaction_matrix(data_aggregated)
    train_data, test_data = hybrid_recsys_train_test_split(interaction_matrix)
    input_dim = interaction_matrix.shape[1]
    encoded_train_data, encoder = hybrid_recsys_train_autoencoder(train_data, test_data, input_dim)
    model = hybrid_recsys_train_xgboost(encoded_train_data, train_data)
    
    users = interaction_matrix.index.tolist()
    recalls, precisions, mses = [], [], []
    
    for user_id in users:
        user_name = user_encoder.inverse_transform([user_id])[0]
        recall, precision, mse = hybrid_recsys_evaluate_recommendations(user_name, user_encoder, item_encoder, interaction_matrix, model, data, encoded_train_data)
        recalls.append(recall)
        precisions.append(precision)
        mses.append(mse)
        print(f"User: {user_name}, Recall@10: {recall:.4f}, Precision@10: {precision:.4f}, MSE: {mse:.4f}")
    
    print(f"\nMean Recall@10: {np.mean(recalls):.4f}")
    print(f"Mean Precision@10: {np.mean(precisions):.4f}")
    print(f"Overall MSE: {np.mean(mses):.4f}")

if __name__ == "__main__":
    main()


Epoch 1/10
76/76 [==============================] - 3s 28ms/step - loss: 2934.1907 - val_loss: 2532.0273
Epoch 2/10
76/76 [==============================] - 1s 18ms/step - loss: 2934.0659 - val_loss: 2531.9937
Epoch 3/10
76/76 [==============================] - 2s 21ms/step - loss: 2934.0400 - val_loss: 2531.9802
Epoch 4/10
76/76 [==============================] - 1s 16ms/step - loss: 2934.0266 - val_loss: 2531.9756
Epoch 5/10
76/76 [==============================] - 1s 13ms/step - loss: 2934.0195 - val_loss: 2531.9722
Epoch 6/10
76/76 [==============================] - 1s 13ms/step - loss: 2934.0142 - val_loss: 2531.9678
Epoch 7/10
76/76 [==============================] - 1s 13ms/step - loss: 2934.0112 - val_loss: 2531.9651
Epoch 8/10
76/76 [==============================] - 1s 13ms/step - loss: 2934.0078 - val_loss: 2531.9661
Epoch 9/10
76/76 [==============================] - 1s 12ms/step - loss: 2934.0073 - val_loss: 2531.9644
Epoch 10/10
303/303 [==============================] - 

User: 51.861.217 ANTONIO MARCOS BATISTA RIBEIRO, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2501.6784
User: 51.862.146 CLAUDIONOR SILVERIO DUTRA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2694.2748
User: 52.045.755 ALESSANDRO DA SILVA SOUSA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 41.6922
User: 52.213.987 EDUARDO SOARES DUARTE, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1328.5474
User: 52.218.817 LUCAS HENRIQUE MENDONCA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 7.8909
User: 52.429.870 DANIEL DOS SANTOS SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 159.4885
User: 52.544.174 WELITON PEREIRA DA ROCHA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 87.8926
User: 52.794.996 JOAO BOSCO MONTEIRO DE MORAES JUNIOR, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 0.9120
User: 52.840.266 LEILA TEREZINHA DE MORAES BERFT, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 231.3862
User: 52.914.265 ISMAEL GOMES GONZAGA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 51.6881
User: 

User: A.C.N. COMERCIO DE GENEROS ALIMENTICIOS LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 3.0048
User: A.F.B. NETO TRANSPORTADORA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2.2403
User: A.G MACAM TRANSPORTES LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 0.3052
User: A.G.S LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 6.0838
User: A.J.B TRANSPORTES RODOVIARIOS LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 682.2282
User: A.M.N. SERVICOS DE COLHEITA LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 982.3913
User: A.M.O TRANSPORTES E LOGISTICA LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 778.4031
User: A.P.F.DOS REIS VEICULOS LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 2.8784
User: A.R TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 3.6551
User: A.S PRODUTORA DE PROTEINA VEGETAL DO TOCANTINS LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 238.4987
User: A.V. TRANSPORTES DE CARGAS E AGENCIAMENTOS LTDA, Recall@10: 0.0000, Pre

User: ADEMIR GONCALVES, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1684.8348
User: ADEMIR JOSE DA ROCHA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2210.0322
User: ADEMIR LOPES DA SILVA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 1000.3442
User: ADEMIR LUIS HERMES, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 57.7039
User: ADEMIR ROSA DANIZETH, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 8.5229
User: ADEMIR SANTIN, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 0.8529
User: ADEMIR VAGNER TEIXEIRA ZAGO, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 505.2492
User: ADEMIR VENANCIO DA SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1550.8555
User: ADEMY DAS CHAGAS SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 16953.2953
User: ADENAER CORENTE, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1.9757
User: ADENILDO LEANDRO DA SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 20.6667
User: ADENILSON BRAZ DE MOURA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 3146.1751
U

User: ADRIANO SOARES DA SILVEIRA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 4294.4542
User: ADRIEL GOUVEIA DE MOURA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2081.2155
User: ADRIEL PEREIRA CURSINO, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 88.2455
User: ADRIEL TEIXEIRA MILIORIM, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2749.9746
User: ADRILOM TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 6.8921
User: ADS MT TRANSPORTES LTDA, Recall@10: 0.3000, Precision@10: 0.3000, MSE: 908.8369
User: ADSERV CONSTRUTORA LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 47.5685
User: ADSON MOURA CUNHA FERREIRA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 4477.0653
User: ADSON TRINDADE DA SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 4008.8336
User: AEDYSSON VIANA DE OLIVEIRA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 25878.6512
User: AEGIX SOLUCOES E LOGISTICA LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 9.0098
User: AELMO OLIVEIRA DA ROCHA, Reca

...

User: RODOFAIXA TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 114.4022
User: RODOFEDERAL TRANSPORTE E LOCACAO LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 1225.8519
User: RODOFIBRA TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 5.7308
User: RODOFILHO TRANSPORTES E LOGISTICA LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 14.4790
User: RODOFRANCO TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 78.6656
User: RODOFREITAS TRANSPORTES LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 0.7228
User: RODOFROTA TRANSPORTES RODOVIARIOS E LOGISTICA LTDA, Recall@10: 0.3000, Precision@10: 0.3000, MSE: 58.9167
User: RODOGALO TRANSPORTES E LOGISTICA LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 873.7361
User: RODOGHEL TRANSPORTES LTDA, Recall@10: 0.2000, Precision@10: 0.2000, MSE: 4080.9432
User: RODOGOMES TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 4.7236
User: RODOGRAO TRANSPORTES LTDA, Recall@10: 0.1000, Preci

User: RODRIGO BARBOSA RODRIGUES, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 3.3211
User: RODRIGO BENITES PAVANI & CIA LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 0.0990
User: RODRIGO BONAPAZ SANTOS, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1732.9173
User: RODRIGO CAZAROTO SOARES, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 3971.4160
User: RODRIGO CESAR FOMIGONI, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1861.8574
User: RODRIGO CESAR RODRIGUES, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 4006.0440
User: RODRIGO CHAVES MACHADO, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 11.6347
User: RODRIGO CORREA & CIA. LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 5.2061
User: RODRIGO CRUZ DA SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2685.8628
User: RODRIGO DA SILVA SOUZA TRANSPORTES RODOVIARIOS, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 13.9223
User: RODRIGO DE ALMEIDA ARRUDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2944.1372
User: RODRIGO DE SOUZ

User: RONALD LIMA DE SOUZA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 4673.1697
User: RONALD RODRIGUES DE ALMEIDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 14.6440
User: RONALDO ALVES DA SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 3463.2391
User: RONALDO ALVES DE SOUZA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1084.8067
User: RONALDO AMORIM DA SILVA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1365.9671
User: RONALDO ANDRADE DE OLIVEIRA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2564.1628
User: RONALDO APARECIDO PAVANI, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 0.3211
User: RONALDO APARECIDO ROCHA ORNELAS, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 0.0282
User: RONALDO APARECIDO ROCHA ORNELAS 00057209154, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 2.7177
User: RONALDO BARBOSA TORRES, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1251.5093
User: RONALDO BENTO MOREIRA JUNIOR, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1.2715
User: RONALDO BUSANEL

User: ROTOPAN TRANSPORTES LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 4274.3521
User: ROTTA GOIAS TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1579.4274
User: ROTTA ZAGO TRANSPORTES LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1.6508
User: ROVER SERVICOS LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 1269.7798
User: ROVISA PECAS E SERVICOS LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 5.6317
User: ROYAL GESSO COMERCIO DE MATERIAIS DE CONSTRUCAO LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 30.2356
User: ROZALINO REZENDE, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2210.4771
User: ROZANE LIRA DOS SANTOS, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 2.7828
User: ROZELIA  DE ALMEIDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 62.0651
User: RP REFORMAS E POSTO DE MOLAS LTDA, Recall@10: 0.0000, Precision@10: 0.0000, MSE: 1.9540
User: RPALOG TRANSPORTES E LOGISTICA LTDA, Recall@10: 0.1000, Precision@10: 0.1000, MSE: 312.5706
User: RPR 

IndexError: index 9686 is out of bounds for axis 0 with size 9686

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from keras.models import Model, Sequential
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
import seaborn as sns
import squarify

# Load the data
file_path = '/Users/igormol/Desktop/Analytics_Fusion/sample/Marajo.csv'
data = pd.read_csv(file_path, parse_dates=['Sales_Date'])

# Preprocess data
data['Sales_Date'] = pd.to_datetime(data['Sales_Date'], errors='coerce')
data.dropna(subset=['Sales_Date'], inplace=True)

# Compute RFM scores
latest_date = data['Sales_Date'].max() + pd.DateOffset(1)
rfm_table = data.groupby('Client_ID').agg({
    'Sales_Date': lambda x: (latest_date - x.max()).days,
    'Client_ID': 'count',
    'Monetary_Value': 'sum'
}).reset_index()
rfm_table.columns = ['Client_ID', 'Recency', 'Frequency', 'Monetary']

# Compute Churn index (hypothetical: 1 if no purchase in the last 90 days, else 0)
rfm_table['Churn'] = rfm_table['Recency'].apply(lambda x: 1 if x > 90 else 0)

# Normalize RFM and churn data
scaler = MinMaxScaler()
rfm_normalized = scaler.fit_transform(rfm_table[['Recency', 'Frequency', 'Monetary', 'Churn']])

# Build and train an autoencoder
input_dim = rfm_normalized.shape[1]
encoding_dim = 2

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(rfm_normalized, rfm_normalized, epochs=50, batch_size=8, shuffle=True, validation_split=0.2)

# Encode the data
encoder_model = Model(inputs=input_layer, outputs=encoder)
encoded_data = encoder_model.predict(rfm_normalized)

# Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(encoded_data)

# Add cluster information to the RFM table
rfm_table['Cluster'] = clusters

# Compute MAE for clustering evaluation
decoded_data = autoencoder.predict(rfm_normalized)
mae = mean_absolute_error(rfm_normalized, decoded_data)
print(f"Mean Absolute Error: {mae}")

# Visualize the clusters using a treemap
cluster_summary = rfm_table.groupby('Cluster').mean().reset_index()
cluster_summary['Count'] = rfm_table['Cluster'].value_counts().values
cluster_summary['Labels'] = ['Cluster ' + str(i) for i in cluster_summary['Cluster']]

plt.figure(figsize=(12, 8))
squarify.plot(sizes=cluster_summary['Count'], label=cluster_summary['Labels'], alpha=.8)
plt.axis('off')
plt.title("Treemap of Clusters")
plt.show()

# Plot mean RFM and churn index for each cluster
cluster_summary.set_index('Labels')[['Recency', 'Frequency', 'Monetary', 'Churn']].plot(kind='bar', figsize=(12, 8))
plt.title("Mean RFM and Churn Index for Each Cluster")
plt.ylabel("Mean Value")
plt.show()


ValueError: cannot insert Client_ID, already exists

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from keras.models import Model
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
import squarify

# Load the data
file_path = '/Users/igormol/Desktop/Analytics_Fusion/sample/Marajo.csv'
data = pd.read_csv(file_path, parse_dates=['Sales_Date'])

# Preprocess data
data['Sales_Date'] = pd.to_datetime(data['Sales_Date'], errors='coerce')
data.dropna(subset=['Sales_Date'], inplace=True)

# Compute RFM scores
latest_date = data['Sales_Date'].max() + pd.DateOffset(1)
rfm_table = data.groupby('Client_ID').agg({
    'Sales_Date': lambda x: (latest_date - x.max()).days,
    'Client_ID': 'count',
    'Monetary_Value': 'sum'
}).reset_index()
rfm_table.columns = ['Client_ID', 'Recency', 'Frequency', 'Monetary']

# Compute Churn index (hypothetical: 1 if no purchase in the last 90 days, else 0)
rfm_table['Churn'] = rfm_table['Recency'].apply(lambda x: 1 if x > 90 else 0)

# Normalize RFM and churn data
scaler = MinMaxScaler()
rfm_normalized = scaler.fit_transform(rfm_table[['Recency', 'Frequency', 'Monetary', 'Churn']])

# Build and train an autoencoder
input_dim = rfm_normalized.shape[1]
encoding_dim = 2

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(rfm_normalized, rfm_normalized, epochs=50, batch_size=8, shuffle=True, validation_split=0.2)

# Encode the data
encoder_model = Model(inputs=input_layer, outputs=encoder)
encoded_data = encoder_model.predict(rfm_normalized)

# Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(encoded_data)

# Add cluster information to the RFM table
rfm_table['Cluster'] = clusters

# Compute MAE for clustering evaluation
decoded_data = autoencoder.predict(rfm_normalized)
mae = mean_absolute_error(rfm_normalized, decoded_data)
print(f"Mean Absolute Error: {mae}")

# Visualize the clusters using a treemap
cluster_summary = rfm_table.groupby('Cluster').mean().reset_index()
cluster_summary['Count'] = rfm_table['Cluster'].value_counts().values
cluster_summary['Labels'] = ['Cluster ' + str(i) for i in cluster_summary['Cluster']]

plt.figure(figsize=(12, 8))
squarify.plot(sizes=cluster_summary['Count'], label=cluster_summary['Labels'], alpha=.8)
plt.axis('off')
plt.title("Treemap of Clusters")
plt.show()

# Plot mean RFM and churn index for each cluster
cluster_summary.set_index('Labels')[['Recency', 'Frequency', 'Monetary', 'Churn']].plot(kind='bar', figsize=(12, 8))
plt.title("Mean RFM and Churn Index for Each Cluster")
plt.ylabel("Mean Value")
plt.show()


ValueError: cannot insert Client_ID, already exists

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from keras.models import Model
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
import squarify

# Load the data
file_path = '/Users/igormol/Desktop/Analytics_Fusion/sample/Marajo.csv'
data = pd.read_csv(file_path, parse_dates=['Sales_Date'])

# Preprocess data
data['Sales_Date'] = pd.to_datetime(data['Sales_Date'], errors='coerce')
data.dropna(subset=['Sales_Date'], inplace=True)

# Compute RFM scores
latest_date = data['Sales_Date'].max() + pd.DateOffset(1)
rfm_table = data.groupby('Client_ID').agg({
    'Sales_Date': lambda x: (latest_date - x.max()).days,
    'Client_ID': 'count',
    'Monetary_Value': 'sum'
}).rename(columns={'Client_ID': 'Frequency'}).reset_index()
rfm_table.columns = ['Client_ID', 'Recency', 'Frequency', 'Monetary']

# Compute Churn index (hypothetical: 1 if no purchase in the last 90 days, else 0)
rfm_table['Churn'] = rfm_table['Recency'].apply(lambda x: 1 if x > 90 else 0)

# Normalize RFM and churn data
scaler = MinMaxScaler()
rfm_normalized = scaler.fit_transform(rfm_table[['Recency', 'Frequency', 'Monetary', 'Churn']])

# Build and train an autoencoder
input_dim = rfm_normalized.shape[1]
encoding_dim = 2

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(rfm_normalized, rfm_normalized, epochs=50, batch_size=8, shuffle=True, validation_split=0.2)

# Encode the data
encoder_model = Model(inputs=input_layer, outputs=encoder)
encoded_data = encoder_model.predict(rfm_normalized)

# Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(encoded_data)

# Add cluster information to the RFM table
rfm_table['Cluster'] = clusters

# Compute MAE for clustering evaluation
decoded_data = autoencoder.predict(rfm_normalized)
mae = mean_absolute_error(rfm_normalized, decoded_data)
print(f"Mean Absolute Error: {mae}")

# Visualize the clusters using a treemap
cluster_summary = rfm_table.groupby('Cluster').mean().reset_index()
cluster_summary['Count'] = rfm_table['Cluster'].value_counts().values
cluster_summary['Labels'] = ['Cluster ' + str(i) for i in cluster_summary['Cluster']]

plt.figure(figsize=(12, 8))
squarify.plot(sizes=cluster_summary['Count'], label=cluster_summary['Labels'], alpha=.8)
plt.axis('off')
plt.title("Treemap of Clusters")
plt.show()

# Plot mean RFM and churn index for each cluster
cluster_summary.set_index('Labels')[['Recency', 'Frequency', 'Monetary', 'Churn']].plot(kind='bar', figsize=(12, 8))
plt.title("Mean RFM and Churn Index for Each Cluster")
plt.ylabel("Mean Value")
plt.show()


Epoch 1/50
1211/1211 [==============================] - 4s 3ms/step - loss: 0.1342 - val_loss: 0.0446
Epoch 2/50
1211/1211 [==============================] - 3s 3ms/step - loss: 0.0162 - val_loss: 0.0032
Epoch 3/50
1211/1211 [==============================] - 3s 2ms/step - loss: 0.0021 - val_loss: 0.0012
Epoch 4/50
1211/1211 [==============================] - 3s 2ms/step - loss: 7.8251e-04 - val_loss: 3.7623e-04
Epoch 5/50
1211/1211 [==============================] - 3s 2ms/step - loss: 3.5738e-04 - val_loss: 2.4386e-04
Epoch 6/50
1211/1211 [==============================] - 3s 2ms/step - loss: 3.0883e-04 - val_loss: 2.3685e-04
Epoch 7/50
1211/1211 [==============================] - 3s 2ms/step - loss: 3.0196e-04 - val_loss: 2.3473e-04
Epoch 8/50
1211/1211 [==============================] - 3s 2ms/step - loss: 2.9703e-04 - val_loss: 2.3147e-04
Epoch 9/50
1211/1211 [==============================] - 3s 3ms/step - loss: 2.9378e-04 - val_loss: 2.2756e-04
Epoch 10/50
1211/1211 [===========

TypeError: Could not convert 2 L TRANSPORTES LTDA2 S TRANSPORTES LTDA23.402.408 IRINEU ACACIO CEZAR2AE AEREO E RODOVIARIO TRANSPORTES LTDA2T TRANSPORTES E LOGISTICA LTDA3 J TRANSPORTES LTDA30.982.398 LUVANIA SOARES FLORIANO31.982.916 NILSON CARDOSO DA SILVA35.655.262 RUIDELMAR GOMES COELHO35.775.673 WENDERSON JOAO PESSOA E SILVA36.796.197 LUCAS SANTOS DA SILVA3B LOGISTICA E TERRAPLANAGEM LTDA3G TRANSPORTES LTDA3M TRANSPORTES LTDA3S TRANSPORTE RODOVIARIO DE CARGA LTDA40.146.358 ADRIANA CAMPOS MATOS44.831.844 NELSON CAMOZI FILHO45.152.233 SIDNEI DA COSTA SILVA49.183.535 EDMILSON ROSA49.186.231 VALDILENE ALEIXA XAVIER GUIMARAES49.218.205 JAIRO ALVES LEITE49.361.480 FLAVIA GRACIELE DE OLIVEIRA MAGNABOSCO49.685.247 GILMAR PEREIRA PORTELA49.689.711 PAULO HENRIQUE FERREIRA DE OLIVEIRA49.722.765 UENDER MACEDO DE OLIVEIRA4E TRANSPORTE DE CARGAS LTDA50.122.965 FLAVIO PIVOTTO50.248.245 ZAQUEU DOS SANTOS CONFORTO50.300.654 EDILSON FERREIRA BARBOSA50.632.873/0001-9550.649.295 JOSE EDUARDO CORREIA H